# Analyse der Internetnutzung und Schülerleistung

In dieser Analyse untersuchen wir den Zusammenhang zwischen der Internetnutzung und der schulischen Leistung von Schülern. Wir betrachten, wie die Nutzung des Internets das Lernverhalten, die Motivation und die Ergebnisse der Schüler beeinflusst.

Durch die Analyse von Daten zur Internetnutzung und schulischen Leistung können wir wichtige Erkenntnisse gewinnen, die uns helfen, Bildungsstrategien zu verbessern und den Schülern dabei zu helfen, ihr volles Potenzial auszuschöpfen.

Wir werden verschiedene Variablen betrachten, wie z.B. die Zeit, die Schüler online verbringen, die Art der Internetnutzung (z.B. für schulische Zwecke oder Freizeitaktivitäten) und die Auswirkungen auf ihre schulischen Leistungen. Darüber hinaus werden wir auch demografische und sozioökonomische Faktoren berücksichtigen, um ein umfassendes Bild zu erhalten.

Zu Beginn werden die Daten geladen und eine explorative Datenanalyse durchgeführt, um die Verteilung der Variablen zu untersuchen und mögliche Zusammenhänge zu identifizieren. Anschließend werden wir verschiedene statistische Methoden anwenden, um den Zusammenhang zwischen Internetnutzung und Schülerleistung zu untersuchen.


TODO score design für bildungs maßnahmen ergriffen von den Schülern selbst

BildungsScore = Internetnutzung * (inf - com - soc) + Bildungsfreizeit + Analoge Inforamtionen

In [19]:
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm
from statsmodels.formula.api import ols

path_prefix2 = 'Anabell/Daten/'
path_prefix = 'Daten/'

# Load the data
pisa_math = pd.read_csv(path_prefix+'pisaMathe.csv', index_col='Jahr', parse_dates=['Jahr'])
pisa_nature = pd.read_csv(path_prefix+'pisaNatur.csv', index_col='Jahr', parse_dates=['Jahr'])
pisa_reading = pd.read_csv(path_prefix+'pisaLese.csv', index_col='Jahr', parse_dates=['Jahr'])

internet_usage = pd.read_csv(path_prefix+'internetNutzung.csv', index_col='Jahr', parse_dates=['Jahr']).xs('Internet Minuten', drop_level=False, axis=1)
print_usage = pd.read_csv(path_prefix+'printNutzung.csv', index_col='Jahr', parse_dates=['Jahr'])
politics = pd.read_csv(path_prefix+'politischesInteresse.csv', index_col='Jahr', parse_dates=['Jahr'])
music_school_attendance = pd.read_csv(path_prefix+'schulerMusikschulen.csv', index_col='Jahr', parse_dates=['Jahr'])
number_of_students = pd.read_csv(path_prefix+'anzahlSchuler.csv', index_col='Jahr', parse_dates=['Jahr'])
#music_schools = pd.read_csv(path_prefix+'musikSchulenAnzahl.csv', index_col='Jahr', parse_dates=['Jahr'])
def pull_inet(num=None):
    if num is None:
        path = 'JIM.csv'
    else:
        path = f'JIM{num}.csv'
    inet_usage_cat = pd.read_csv(path_prefix+path, index_col=['Aktivität','Kategorie']).groupby('Kategorie').sum()
    inet_usage_cat = inet_usage_cat.T.sort_index()
    inet_usage_cat.index.name = 'Jahr'
    if 'Soziales Netzwerk' not in inet_usage_cat.columns:
        inet_usage_cat['Soziales Netzwerk'] = 0
    # extend index to 2000
    new_idx = [str(i) for i in range(2000, 2024)]
    # set 0 to na
    inet_usage_cat = inet_usage_cat.replace(0, np.nan)
    inet_usage_cat = inet_usage_cat.reindex(new_idx).bfill()
    inet_usage_cat.index = pd.to_datetime(inet_usage_cat.index)
    # add suffix if not none
    if num is not None:
        inet_usage_cat = inet_usage_cat.add_suffix(f' {num}')
    return inet_usage_cat

inet_usage_cat = pull_inet()
inet_usage_cat2 = pull_inet(2)

inet_usage_cat3_raw = pd.read_csv(path_prefix+'JIM3.csv')
inet_map = inet_usage_cat3_raw.loc[:,['Kategorie1', 'Kategorie2']].groupby('Kategorie1').first()
inet_usage_cat3 = inet_usage_cat3_raw.set_index(['Aktivität','Kategorie1','Kategorie2']).groupby('Kategorie1').max()
inet_usage_cat = pd.concat([inet_usage_cat3, inet_map], axis=1).groupby('Kategorie2').sum().T.sort_index()
inet_usage_cat.index.name = 'Jahr'
if 'Soziales Netzwerk' not in inet_usage_cat.columns:
    inet_usage_cat['Soziales Netzwerk'] = 0
# extend index to 2000
new_idx = [str(i) for i in range(2000, 2024)]
# set 0 to na
inet_usage_cat = inet_usage_cat.replace(0, np.nan)
inet_usage_cat = inet_usage_cat.reindex(new_idx).bfill()
inet_usage_cat.index = pd.to_datetime(inet_usage_cat.index)

# idea: the upraising pisa scores where interupted by the introduction of social media
# TODO: anzahl schüler im verältnis zu schülerzahl insgesamt
# TODO: Delta score vllt besser um impact zu zeigen


In [21]:
# Merge the data
data_raw = pd.concat([pisa_math, pisa_nature, pisa_reading, print_usage, politics, music_school_attendance, internet_usage, inet_usage_cat, inet_usage_cat2,number_of_students], axis=1)
data = data_raw.sort_index().ffill().bfill()
data = data.loc['2000':]

In [22]:
data['Internet Gewicht'] = data['Internet Minuten'] / data['Internet Minuten'].loc['2015'].values
data['Info Score Internet'] = data['Information'] / (data['Kommunikation'] + data['Unterhaltung'] + data['Soziales Netzwerk'] + data['Information']) * data['Internet Gewicht']
data['Info Score Print'] = (data['Tageszeitung'] + data['Zeitschriften']) / (data['Bücher'] + data['Tageszeitung'] + data['Zeitschriften'])
data['Hobby Score'] = data['Musikschüler'] / data['Schüler in Deutschland']
data['Bildungs Score'] = (data['Info Score Internet'] + data['Info Score Print'] + data['Hobby Score']) / 3
data['PISA Score'] = (data['Mathematik'] + data['Naturwissenschaften'] + data['Lesekompetenz']) / 3

data['Info Score Internet 2'] = data['Information 2'] / (data['Kommunikation 2'] + data['Unterhaltung 2'] + data['Information 2']) * data['Internet Gewicht']

In [23]:
fig = px.line(data, y=['Information', 'Kommunikation','Soziales Netzwerk', 'Unterhaltung'], template='plotly_white')
fig.update_layout(title='Internetnutzung kumulierte Häufigkeiten',autosize=False, width=800,height=600)
fig.update_yaxes(title='Kumulierte Häufigkeit')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=0.9,
    title=''
))
# color template
fig.show()

In [6]:
fig = px.line(data, y=['Information 2', 'Kommunikation 2','Soziales Netzwerk 2', 'Unterhaltung 2'], template='plotly_white')
fig.update_layout(title='EXPERIMENT: Internetnutzung kumulierte Häufigkeiten',autosize=False, width=800,height=600)
fig.update_yaxes(title='Kumulierte Häufigkeit')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=0.9,
    title=''
))
# color template
fig.show()

In [7]:
fig = px.line(data, y=['Bücher','Zeitschriften','Tageszeitung'], template='plotly_white')
fig.update_layout(title='Printmedien Nutzung mehrmals die Woche (%)',autosize=False, width=800,height=600)
fig.update_yaxes(title='Prozent')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=0.75,
    title=''
))
fig.show()

In [8]:
fig = px.line(data, y=['Schüler in Deutschland', 'Musikschüler'], template='plotly_white')
fig.update_layout(title='Anzahl Schüler und Musikschüler',autosize=False, width=800,height=600)
fig.update_yaxes(title='Anzahl')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=0.75,
    title=''
))
fig.show()

In [26]:
fig = px.line(data, y=['Info Score Internet', 'Info Score Print', 'Hobby Score', 'Bildungs Score'], template='plotly_white')
fig.update_layout(title='Scores über die Zeit',autosize=False, width=800,height=600)
fig.update_yaxes(title='Score')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=0.9,
    title=''
))
fig.show()

In [10]:
fig = px.imshow(data.loc[:,['Mathematik', 'Naturwissenschaften', 'Lesekompetenz','Info Score Internet', 'Info Score Print', 'Hobby Score']].corr())
fig.update_layout(title='Correlation Matrix',autosize=False, width=800,height=600)
fig.show()

In [11]:
# plot pisa
fig = px.line(data, y=['Mathematik', 'Naturwissenschaften', 'Lesekompetenz'], template='plotly_white')
fig.update_layout(title='PISA Ergebnisse 2000-2024',autosize=False, width=800,height=600)
fig.update_yaxes(title='Punkte')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=0.9,
    title=''
))
fig.show()

In [24]:
plt_data = data.loc[:,['PISA Score', 'Bildungs Score']]
plt_data = plt_data / plt_data.loc['2000'].iloc[0]
fig = px.line(plt_data, y=['PISA Score', 'Bildungs Score'], template='plotly_white')
fig.update_layout(title='Score Vergleiche',autosize=False, width=800,height=600)
fig.update_yaxes(title='Normalisierter Score')
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=0.7,
    title=''
))
fig.show()

In [25]:
# fit a linear model with formula
reg_data = data.loc[:,['PISA Score', 'Bildungs Score']]
reg_data.columns = ['PISA_Score', 'Bildungs_Score']
formula = 'PISA_Score ~ Bildungs_Score'
model = ols(formula, reg_data).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             PISA_Score   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     14.01
Date:                 So, 07 Jul 2024   Prob (F-statistic):            0.00113
Time:                        20:16:00   Log-Likelihood:                -83.146
No. Observations:                  24   AIC:                             170.3
Df Residuals:                      22   BIC:                             172.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        425.3849     20.422     20.829      0.000     383.031     467.739
Bildungs_Score   228.3891     61.012      3.743      0.001     101.857     354.921
==============================================================================
Omnibus:                        0.979   Durbin-Watson:                   0.462
Prob(Omnibus):                  0.613   Jarque-Bera (JB):                0.891
Skew:                          -0.257   Prob(JB):                        0.641
Kurtosis:                       2.209   Cond. No.                         41.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [122]:
print(model.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &   PISA\_Score   & \textbf{  R-squared:         } &     0.581   \\
\textbf{Model:}            &       OLS       & \textbf{  Adj. R-squared:    } &     0.562   \\
\textbf{Method:}           &  Least Squares  & \textbf{  F-statistic:       } &     30.50   \\
\textbf{Date:}             & Do, 04 Jul 2024 & \textbf{  Prob (F-statistic):} &  1.50e-05   \\
\textbf{Time:}             &     23:23:59    & \textbf{  Log-Likelihood:    } &   -78.622   \\
\textbf{No. Observations:} &          24     & \textbf{  AIC:               } &     161.2   \\
\textbf{Df Residuals:}     &          22     & \textbf{  BIC:               } &     163.6   \\
\textbf{Df Model:}         &           1     & \textbf{                     } &             \\
\textbf{Covariance Type:}  &    nonrobust    & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                         & \textbf{coef} & \text